<a href="https://colab.research.google.com/github/KILjungjoon/Project_Korean/blob/main/%ED%8F%89%EA%B0%80%EB%8D%B0%EC%9D%B4%ED%84%B0(%ED%91%9C%EC%A4%80%EB%B2%84%EC%A0%84)%EC%96%B4%ED%9C%98%EC%8C%8D%EA%B3%BC_%EC%9D%BC%EC%B9%98%ED%95%98%EB%8A%94_w2v%EC%96%B4%ED%9C%98%EC%8C%8D_%EC%B0%BE%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
# 평가데이터(표준버전)어휘쌍 파일 읽기
eval = pd.read_csv('/content/drive/MyDrive/KoreanProject/가공data/words_grade_표준ver.csv')

In [ ]:
# w2v 모델을 불러온다.
from gensim.models import word2vec
model=word2vec.Word2Vec.load('/content/drive/MyDrive/KoreanProject/w2v_100vec_1min_5wind.model')

In [ ]:
# 시간 측정 라이브러리 install
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tqdm import tqdm
from time import sleep
import numpy as np
import os

chunk_size = 5000   # 5천행씩 나눠서 실행
eval_chunks = np.array_split(eval, np.ceil(len(eval) / chunk_size))

for i, eval_chunk in tqdm(enumerate(eval_chunks)):
    result = []
    for j, row in eval_chunk.iterrows():
        try:
            similar_words = [(word, similarity) for word, similarity in model.most_similar(row['head.의미어'], topn=1000000) if similarity > 0.5]
            vec = pd.DataFrame(similar_words, columns=['word', 'similarity'])
            if row['tail.의미어'] in vec['word'].values:
                result.append([row['head.의미어'], row['tail.의미어'], row['beta'], vec[vec['word'] == row['tail.의미어']]['similarity'].values[0]])
        except KeyError:
            continue
    result = pd.DataFrame(result, columns=['head.의미어', 'tail.의미어',  'beta', 'similarity'])
    filename = "/content/drive/MyDrive/KoreanProject/가공data/result/result_{}.csv".format(i)
    result.to_csv(filename, index=False)

0it [00:00, ?it/s]<ipython-input-7-74c97d9385b3>:14: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  similar_words = [(word, similarity) for word, similarity in model.most_similar(row['head.의미어'], topn=1000000) if similarity > 0.5]
9it [2:15:32, 903.56s/it]


In [ ]:
# 최종 결과 파일 불러와서 확인하기
df = pd.read_csv('/content/drive/MyDrive/KoreanProject/가공data/result/sg5win100vec1min_result.csv')
df.shape

(26498, 4)

---
---
---
* 이하 소스 코드

In [ ]:
# 폴더 안의 모든 csv파일을 하나로 합치기
import datetime
import os
import pandas as pd
start=datetime.datetime.now()
# Get a list of all csv files in the folder
folder_path = "/content/drive/MyDrive/KoreanProject/가공data/result"
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
csv_files.sort()

# Read each csv file into a pandas DataFrame
df_list = [pd.read_csv(os.path.join(folder_path, f)) for f in csv_files]

# Concatenate the DataFrames vertically (row-wise)
result = pd.concat(df_list)

# Write the result to a single csv file
result.to_csv("/content/drive/MyDrive/KoreanProject/가공data/result/sg5win100vec1min_result.csv", index=False)
end=datetime.datetime.now()
print(end-start)

0:00:12.584700


In [ ]:
# chunk로 나누지 않고 실행할 때 사용하는 코드
result = []
for i, row in eval.iterrows():
    try:
        similar_words = [(word, similarity) for word, similarity in model.most_similar(row['head.의미어'], topn=1000000) if similarity > 0.5]
        vec = pd.DataFrame(similar_words, columns=['word', 'similarity'])
        if row['tail.의미어'] in vec['word'].values:
            result.append([row['head.의미어'], row['tail.의미어'], row['beta'], vec[vec['word'] == row['tail.의미어']]['similarity'].values[0]])
    except KeyError:
        continue
result = pd.DataFrame(result, columns=['head.의미어', 'tail.의미어',  'beta', 'similarity'])
filename = "/content/drive/MyDrive/KoreanProject/가공data/result/result_{}.csv".format(i)
result.to_csv(filename, index=False)

0it [00:00, ?it/s]<ipython-input-15-cd0b2466a863>:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  similar_words = [(word, similarity) for word, similarity in model.most_similar(row['head.의미어'], topn=1000000) if similarity > 0.5]
9658it [20:36, 10.30it/s]

In [ ]:
## x가 주어졌을 때 similar_words와 유사도를 구하는 소스 코드

# Find words similar to x with a similarity of 0.5 or higher
x = "내벽"
# Get the top 10 similar words with similarity greater than 0.5
similar_words = [(word, similarity) for word, similarity in model.most_similar(x, topn=1000000) if similarity > 0.5]   # topn=1000000, 유사도> 0.5
# Create a dataframe with words and similarity as columns
df = pd.DataFrame(similar_words, columns=['word', 'similarity'])
# Print the dataframe
df

<ipython-input-18-cc4d0f260381>:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  similar_words = [(word, similarity) for word, similarity in model.most_similar(x, topn=100000) if similarity > 0.5]


,word,similarity
0,살덩이,0.892290
1,얼음장,0.888160
2,화기02,0.886449
3,꾸물꾸물,0.885672
4,질퍽하다,0.883920
...,...,...
60579,지대09,0.500046
60580,산철,0.500034
60581,상채기,0.500028
60582,전운01,0.500025


In [ ]:
model.most_similar("천국01")[0]

<ipython-input-12-ef8acf62456e>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("천국01")[0]


('낙원', 0.7937644720077515)

In [ ]:
df[df['word'].str.contains('안벽')]

,word,similarity
14598,안벽,0.71078


In [ ]:
df[df['head.의미어'].str.contains('건축값')]

,head.voca,head.의미어,head.의미번호,tail.voca,tail.의미어,tail.의미번호,beta
0,건축-값,건축값001,1,건축-액,건축액001,1,2.620510
79005,건축-값,건축값001,1,비용,비용004,4,3.022731
180004,건축-값,건축값001,1,공사-비,공사비001,1,3.054716
